In [ ]:
pip install transformers datasets torch tqdm peft bitsandbytes accelerate autoawq compressed-tensors openpyxl evaluate rouge_score

In [ ]:
import openpyxl

def read_xlsx_to_nested_list(file_path):
    all_lists = []
    current_list = []
    headers_length = 0

    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    for row in sheet.iter_rows(values_only=True):
        if all(cell is None for cell in row):
            if current_list:
                all_lists.append(current_list)
                current_list = []
            continue

        if row[0] == "Полное название":
            current_list.append([cell for cell in row if cell is not None])
            headers_length = len(current_list[0])
        else:
            if current_list:
                current_row = []
                for i in range(headers_length):
                    if i < len(row) and row[i] is not None:
                        current_row.append(row[i])
                    else:
                        current_row.append('')
                current_list.append(current_row)

    if current_list:
        all_lists.append(current_list)

    all_lists = [lst for lst in all_lists if lst]

    return all_lists

file_path = '/content/drive/MyDrive/raw_data_generated.xlsx'
result = read_xlsx_to_nested_list(file_path)

def format_data_to_dict(nested_list):
    result = []

    for sublist in nested_list[1:]:
        question_text = sublist[0]

        answer_parts = []
        for i in range(1, len(sublist)):
            header = nested_list[0][i]
            value = sublist[i]
            answer_parts.append(f"{header}:{value}")

        answer_text = "/sprt/".join(answer_parts)

        result.append({
            "input": question_text,
            "output": answer_text
        })

    return result

resulted = []
for array in result:
    a = format_data_to_dict(array)
    for larray in a:
        resulted.append(larray)

res_len = int(len(resulted))
print(res_len)
data1 = resulted[0:300:3]
data2 = resulted[1:300:3]
data = data1 + data2
validation_data = resulted[2:300:3]

# Оставляем закомментированные варианты без изменений
# data1 = resulted[200:600:3]
# data2 = resulted[201:600:3]
# data = data1 + data2
# validation_data = resulted[202:600:3]

# data1 = resulted[0:78:3]
# data2 = resulted[1:78:3]
# data = data1 + data2
# validation_data = resulted[2:78:3]

print(data)
print(validation_data)

1500
[{'input': 'Автошина 205 мм, 55%', 'output': 'Категория:Автошина/sprt/Ширина:205 мм/sprt/Высота:55%/sprt/Радиус:/sprt/Сезонность:/sprt/Шипованная:'}, {'input': 'Автошина 60%, нешипованная, R15, всесезонная, 245 мм', 'output': 'Категория:Автошина/sprt/Ширина:245 мм/sprt/Высота:60%/sprt/Радиус:R15/sprt/Сезонность:всесезонная/sprt/Шипованная:нешипованная'}, {'input': 'Автошина R16, летняя, шипованная, 245 мм', 'output': 'Категория:Автошина/sprt/Ширина:245 мм/sprt/Высота:/sprt/Радиус:R16/sprt/Сезонность:летняя/sprt/Шипованная:шипованная'}, {'input': 'Автошина нешипованная, R15, 75%, 245 мм', 'output': 'Категория:Автошина/sprt/Ширина:245 мм/sprt/Высота:75%/sprt/Радиус:R15/sprt/Сезонность:/sprt/Шипованная:нешипованная'}, {'input': 'Автошина шипованная', 'output': 'Категория:Автошина/sprt/Ширина:/sprt/Высота:/sprt/Радиус:/sprt/Сезонность:/sprt/Шипованная:шипованная'}, {'input': 'Автошина 225 мм, 70%, всесезонная, R16, шипованная', 'output': 'Категория:Автошина/sprt/Ширина:225 мм/sprt/Выс

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import numpy as np
import evaluate
import math
import os

MODEL_NAME = "TheBloke/Llama-2-7B-Chat-AWQ"
MODEL_SAVE_PATH = "./drive/MyDrive/pp_4sem/fine_tuned_model"

dataset = Dataset.from_list(data)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    inputs = [
        f"Вопрос: {inp}\nОтвет: {out}"
        for inp, out in zip(examples["input"], examples["output"])
    ]
    tokens = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens



tokenized_datasets = dataset.map(tokenize_function, batched=True)

validation_dataset = Dataset.from_list(validation_data)
tokenized_validation_datasets = validation_dataset.map(tokenize_function, batched=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cuda:0"
)

# if os.path.exists(MODEL_SAVE_PATH):
#     print("Загружаем дообученную модель...")
#     model = AutoModelForCausalLM.from_pretrained(
#         MODEL_SAVE_PATH,
#         torch_dtype=torch.float16,
#         device_map="cuda:0"
#     )
#     learning_rate = 2e-5
# else:
#     print("Загружаем исходную модель...")
#     model = AutoModelForCausalLM.from_pretrained(
#         MODEL_NAME,
#         torch_dtype=torch.float16,
#         device_map="cuda:0"
#     )
#     learning_rate = 5e-4

lora_config = LoraConfig(
    r=2,
    lora_alpha=4,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Метрики
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

# Добавляем кастомные метрики
def extract_key_value_pairs(text):
    """Парсит строку вида 'ключ1:значение1/sprt/ключ2:значение2' в словарь"""
    pairs = text.split('/sprt/')
    result = {}
    for pair in pairs:
        if ':' in pair:
            key, value = pair.split(':', 1)
            result[key.strip()] = value.strip()
    return result

def calculate_metrics(pred_dict, true_dict):
    """Вычисляет TP, FP, FN для пар ключ-значение"""
    tp = 0  # Полностью совпадающие пары
    fp = 0  # Лишние пары в предсказании
    fn = 0  # Пропущенные пары из истинных данных

    for key, true_value in true_dict.items():
        pred_value = pred_dict.get(key)
        if pred_value == true_value:
            tp += 1
        else:
            fn += 1

    for key in pred_dict:
        if key not in true_dict:
            fp += 1

    return tp, fp, fn

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Перплексия
    logits_tensor = torch.tensor(logits)
    labels_tensor = torch.tensor(labels)
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(logits_tensor.view(-1, logits_tensor.size(-1)), labels_tensor.view(-1))
    perplexity = math.exp(loss.item())

    # Декодируем тексты
    pred_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Вычисляем F1 и Key-Value Accuracy
    total_tp = total_fp = total_fn = 0
    kv_accuracy_count = 0

    for pred_text, true_text in zip(pred_texts, label_texts):
        pred_dict = extract_key_value_pairs(pred_text)
        true_dict = extract_key_value_pairs(true_text)

        # Считаем полностью совпадающие пары
        if pred_dict == true_dict:
            kv_accuracy_count += 1

        # Считаем TP/FP/FN
        tp, fp, fn = calculate_metrics(pred_dict, true_dict)
        total_tp += tp
        total_fp += fp
        total_fn += fn

    # Рассчитываем Precision, Recall, F1
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Key-Value Accuracy
    kv_accuracy = kv_accuracy_count / len(pred_texts) if len(pred_texts) > 0 else 0

    # Существующие метрики
    rouge_scores = rouge_metric.compute(predictions=pred_texts, references=label_texts)
    meteor_score = meteor_metric.compute(predictions=pred_texts, references=label_texts)

    return {
        "perplexity": perplexity,
        "rouge-L": rouge_scores["rougeL"],
        "meteor": meteor_score["meteor"],
        "kv_precision": precision,
        "kv_recall": recall,
        "kv_f1": f1,
        "kv_accuracy": kv_accuracy
    }

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/pp_4sem/results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=15,
    learning_rate=5e-4,
    lr_scheduler_type="cosine",
    fp16=True,
    logging_dir="./logs",
    logging_strategy="epoch",
    report_to="tensorboard",
    save_total_limit=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="rouge-L",
    greater_is_better=True,
    # warmup_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_validation_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
trainer.save_model(MODEL_SAVE_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-9fb1afd3cd47>:186: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Perplexity,Rouge-l,Meteor,Kv Precision,Kv Recall,Kv F1,Kv Accuracy
1,1.692600,0.345332,141087.397679,0.783105,0.650062,0.662708,0.473684,0.552475,0.000000
2,0.230300,0.159483,807907.148330,0.802148,0.790244,0.826087,0.806452,0.816151,0.000000
3,0.104500,0.087586,2533840.989758,0.848063,0.840898,1.000000,0.830221,0.907236,0.000000
4,0.084600,0.088378,3867354.670357,0.855282,0.829884,1.000000,0.830221,0.907236,0.000000
5,0.080200,0.086289,4952794.367230,0.843267,0.842292,1.000000,0.830221,0.907236,0.000000
6,0.076200,0.087291,4766041.089205,0.847102,0.848746,1.000000,0.830221,0.907236,0.000000


In [ ]:
# # это не трогайте
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
# from peft import LoraConfig, get_peft_model
# from datasets import Dataset
# import numpy as np
# import evaluate
# import math
# import os

# MODEL_NAME = "TheBloke/Llama-2-7B-Chat-AWQ"
# MODEL_SAVE_PATH = "./drive/MyDrive/pp_4sem/fine_tuned_model"

# dataset = Dataset.from_list(data)

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

# def tokenize_function(examples):
#     tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
#     tokens["labels"] = tokens["input_ids"].copy()
#     return tokens

# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# validation_dataset = Dataset.from_list(validation_data)
# tokenized_validation_datasets = validation_dataset.map(tokenize_function, batched=True)

# training_args = TrainingArguments(
#     output_dir="./drive/MyDrive/pp_4sem/results",
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
#     gradient_accumulation_steps=4,
#     num_train_epochs=15,
#     learning_rate=5e-4,
#     lr_scheduler_type="cosine",
#     fp16=True,
#     logging_dir="./logs",
#     report_to="none",
#     save_total_limit=1,
#     evaluation_strategy="epoch",
#     save_strategy="epoch"
# )

# lora_config = LoraConfig(
#     r=8,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16,
#     device_map="cuda:0"
# )


# # if os.path.exists(MODEL_SAVE_PATH):
# #     print("Загружаем дообученную модель...")
# #     model = AutoModelForCausalLM.from_pretrained(
# #         MODEL_SAVE_PATH,
# #         torch_dtype=torch.float16,
# #         device_map="cuda:0"
# #     )
# #     training_args.learning_rate = 5e-5
# # else:
# #     print("Загружаем исходную модель...")
# #     model = AutoModelForCausalLM.from_pretrained(
# #         MODEL_NAME,
# #         torch_dtype=torch.float16,
# #         device_map="cuda:0"
# #     )


# model = get_peft_model(model, lora_config)

# accuracy_metric = evaluate.load("accuracy")
# rouge_metric = evaluate.load("rouge")
# bleu_metric = evaluate.load("bleu")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)

#     logits = torch.tensor(logits)
#     labels = torch.tensor(labels)

#     mask = labels != -100

#     flattened_predictions = predictions[mask].flatten()
#     flattened_labels = labels[mask].flatten()
#     accuracy = accuracy_metric.compute(predictions=flattened_predictions.tolist(), references=flattened_labels.tolist())["accuracy"]

#     loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
#     loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
#     perplexity = math.exp(loss.item())

#     pred_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     rouge_scores = rouge_metric.compute(predictions=pred_texts, references=label_texts)

#     bleu_score = bleu_metric.compute(predictions=pred_texts, references=[[ref] for ref in label_texts])

#     return {
#         "accuracy": accuracy * 100,
#         "perplexity": perplexity,
#         "rouge-L": rouge_scores["rougeL"],
#         "bleu": bleu_score["bleu"]
#     }

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets,
#     eval_dataset=tokenized_validation_datasets,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# trainer.train()

# trainer.save_model("./drive/MyDrive/pp_4sem/fine_tuned_model")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

<ipython-input-3-e315d3ccfe3f>:113: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Perplexity,Rouge-l,Bleu
1,No log,0.298532,42.859375,18822.311612,0.748661,0.654879
2,No log,0.152100,42.789062,71219.640059,0.837504,0.836787
3,No log,0.099972,42.734375,131443.717842,0.827828,0.835989
4,No log,0.091712,42.804688,170580.360008,0.833701,0.844658
5,No log,0.091180,42.726562,227482.885078,0.839529,0.846745
6,No log,0.091637,42.679688,286548.025786,0.844209,0.850741
7,No log,0.095935,42.726562,295900.438502,0.839489,0.851361
8,No log,0.095018,42.703125,351632.834662,0.843007,0.850934
9,No log,0.095437,42.703125,368208.042016,0.844706,0.848842
10,No log,0.102870,42.742188,418760.339961,0.840234,0.855496


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import evaluate

MODEL_PATH = "./drive/MyDrive/pp_4sem/fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH).to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# def generate_answer(input_text):
#     prompt = f"<start>\nINPUT: {input_text}\nOUTPUT:"
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=64,
#         do_sample=False,
#         temperature=0.7,
#         top_p=0.9,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id
#     )
#     decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return decoded.split("OUTPUT:")[-1].replace("<end>", "").strip()

# print(generate_answer("Амперметр 0.1%, 220V, 0-10А\nОтвет:"))

def generate_with_few_shot(model, tokenizer, input_text, examples, max_new_tokens=256):
    few_shot_prompt = ""
    for ex in examples:
        few_shot_prompt += f"INPUT: {ex['input']}\nOUTPUT: {ex['output']}{tokenizer.eos_token}\n\n"

    few_shot_prompt += f"INPUT: {input_text}\nOUTPUT:"

    inputs = tokenizer(few_shot_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Удаляем few-shot префикс
    result = decoded[len(few_shot_prompt):].strip()

    # Обрезаем всё, что после следующего 'INPUT:' (если вдруг модель продолжила)
    result = result.split("INPUT:")[0].strip()

    return result

examples = [
    {"input": "Автошина 205 мм, 55%, R1", "output": "Категория:Автошина/sprt/Ширина:205 мм/sprt/Высота:55%/sprt/Радиус:R1/sprt/Сезонность:/sprt/Шипованная:"},
    {'input': 'Аккумулятор 24V, прямая, евро', 'output': 'Категория:Аккумулятор/sprt/Емкость:/sprt/Напряжение:24V/sprt/Тип:/sprt/Полярность:прямая/sprt/Размер:евро'},
    {'input': 'Баллон газовый алюминий, 10л', 'output': 'Категория:Баллон газовый/sprt/Объем:10л/sprt/Давление:/sprt/Материал:алюминий/sprt/Тип:'},
    {'input': 'Блок питания 750 Вт, 80 Plus Platinum, ATX', 'output': 'Категория:Блок питания/sprt/Мощность:750 Вт/sprt/Сертификация:80 Plus Platinum/sprt/Разъем:ATX'},
    {'input': 'Генератор 220V', 'output': 'Категория:Генератор/sprt/Мощность:/sprt/Напряжение:220V/sprt/Топливо:/sprt/Тип:'}
]

test_input = "Генератор 120V, бензиновый"

result = generate_with_few_shot(model, tokenizer, test_input, examples)
print("Generated:\n", result)



# input_text = "Амперметр 0.1%, 220V, 0-10А\nОтвет:"
# etalon_answer = "Категория:Амперметр/sprt/Диапазон:0-10А/sprt/Точность:0.1%/sprt/Тип:/sprt/Питание:220V"
# inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
# output = model.generate(**inputs, max_new_tokens=100)
# response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

# print("Сгенерированный ответ (repr):", repr(response))
# print("Эталонный ответ (repr):", repr(etalon_answer))
# print("Строки равны?:", response == etalon_answer)

# rouge_metric = evaluate.load("rouge")
# rouge_scores = rouge_metric.compute(predictions=[response], references=[etalon_answer])

# print("Сгенерированный ответ:", response)
# print("Эталонный ответ:", etalon_answer)
# print("ROUGE-L:", rouge_scores["rougeL"])

# # llama 3.1 8b instruct
# # температуру

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Сгенерируй текст аналогично моему примеру: Вопрос: Автошина 185 мм, всесезонная, R18
Ответ:; Ответ: Автошина 185 мм, всесезонная, R18
Ответ: Категория:Автошина/sprt/Ширина:185 мм/sprt/Высота:/sprt/Радиус:R18/sprt/Сезонность:всесезонная/sprt/Шипованная:. Надо сгенерировать точно так же для следуещего вопроса: Автошина 185 мм, всесезонная, R123, шипованная
Ответ: Категория:Автошина/sprt/Ширина:185 мм/sprt/Высота:/sprt/Радиус:R123/sprt/Сезонность:всесезонная/sprt/Шипованная:.
